In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise04AI import Exercise04AI
from Exercise06AI import Exercise06AI

# Aufgabe 07: Minimax mit Alpha-Beta-Pruning, Memoisierung und Progressive Deepening

Dieses Notebook erweitert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung um das Progressive Deepening.

In [ ]:
import chess
from typing import Any


class Exercise07AI(Exercise06AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning, memoization and progressive deepening"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

## Progressive Deepening

Das in `Exercise05AI` implementierte Alpha-Beta-Pruning ist deutlich effizienter (aber nicht zwingend optimal), wenn zuerst die besten Züge untersucht werden [9]. Aus diesem Grund wird nun die Liste der möglichen Züge vorab nach der bisher bekannten Evaluierung jedes Zuges sortiert. Die bisherigen Evaluierungen werden dabei aus dem, in `Exercise06AI` implementierten, Memoisierungs-Cache abgerufen. Wurde ein Zug bereits evaluiert, so wird das Tripel $(\textrm{flag}, v, m)$ aus dem Cache zurückgegeben, wobei $v$ eine Annäherung an die Evaluierung des Zuges und $m$ der Zug ist.

Die iterative Tiefensuche wurde ursprünglich verwendet, um die grundlegende zeitliche Kontrolle über Programm zu erhalten, da nach jeder Iteration die Suche abgebrochen werden kann, sofern ein zeitliches Limit überschritten wurde. In Kombination mit Alpha-Beta-Pruning ist die iterative Tiefensuche jedoch schneller als wenn direkt nach dem Ergebnis gesucht wird, da durch die Sortierung der Züge das Alpha-Beta-Fenster schneller verkleinert und somit mehr Knoten im Suchbaum übersprungen werden können[9].

Wie in [10] beschrieben, werden beim Progressive Deepening die Züge iterativ bis zu einer gegebenen Tiefe `DEPTH` berechnet. Dabei wird mit der Tiefe `1` begonnen um eine grobe Wertung der bis dahin möglichen Züge zu erhalten. Beim nächsten Aufruf mit Tiefe `2` kann dann die vorherige Evaluierung genutzt werden, um die Züge anhand diesem Maß zu sortieren und somit das Pruning signifikant zu erhöhen.

Die Funktion `minimax` wird um eine mit [heapq](https://docs.python.org/3/library/heapq.html) implementierte Prioritäts-Warteschlange `moves` erweitert, welche alle verfügbaren Züge speichert. Als Priorität wird hierbei die Evaluierung des Zuges mit geringerer Tiefe verwendet. Falls keine Evaluierung vorhanden ist, wird eine fortlaufende Zugnummer als Priorität verwendet. Bedingt durch die Funktionsweise der Sortierung von `heapq` wird nun ein Tripel der Form

$$ \textrm{key} = (\textrm{evaluation}, \textrm{i}, \textrm{move}) $$

erstellt. Hierbei wird für den weißen Spieler $evaluation$ invertiert, da bei `heapq` der niedrigste Wert die höchste Priorität hat und bei der Maximierung eine umgekehrte Priorisierung erforderlich ist. Zusätzlich wird an zweiter Position eine fortlaufende Zugnummer $i$ hinzufügt. Dies ist notwendig, da die Evaluierung für zwei Züge gleich sein kann und `heapq` in diesem Fall die nächsten Elemente des Tupels vergleicht. Für $move$ ist das nicht möglich, daher wird zuvor $i$ eingefügt um diesen Vergleich (und die ansonsten entstehende `Exception`) zu verhindern. Die Berechnung der neuen Evaluierung erfolgt nun in zwei Schritten:

1. Die in `board.legal_moves` verfügbaren Züge werden iterativ gegen den Cache geprüft. Wenn die Evaluierung des Zuges bereits mit einer Tiefe von `depth - 2` verfügbar ist, wird diese genutzt, ansonsten wird die Zugnummer $i$ verwendet. Anschließend wird der Zug mit dem erhaltenen Wert (evtl. invertiert) als Priorität in die Warteschlange einsortiert.
2. An dieser Stelle sind alle möglichen Züge bereits in der Liste `moves` enthalten und der Zug mit der bisher besten Evaluierung kann mithilfe von `heapq.heappop(moves)[2]` extrahiert werden.

In [ ]:
from typing import Any
import heapq


class Exercise07AI(Exercise07AI):  # type: ignore
    @Exercise06AI.memoize_minimax
    def minimax(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise07AI.LIMIT,
        beta: int = Exercise07AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm"""
        self.stats[-1]["nodes"] += 1
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            return early_abort_evaluation, None

        best_move = None

        # White to play (positive numbers are good)
        moves: list[tuple[int, int, chess.Move]] = []
        if board.turn:
            for i, move in enumerate(board.legal_moves):
                board.push(move)
                key = Exercise04AI.get_key(board, depth - 2)
                board.pop()
                old_eval = self.cache.get(key, (None, i, None))[1]
                heapq.heappush(moves, (-old_eval, i, move))
            maxEvaluation = alpha
            while moves:
                move = heapq.heappop(moves)[2]
                new_evaluation = self.incremental_evaluate(
                    board, current_evaluation, move
                )
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    new_evaluation,
                    maxEvaluation,
                    beta,
                )
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            for i, move in enumerate(board.legal_moves):
                board.push(move)
                key = Exercise04AI.get_key(board, depth - 2)
                board.pop()
                old_eval = self.cache.get(key, (None, i, None))[1]
                heapq.heappush(moves, (old_eval, i, move))
            minEvaluation = beta
            while moves:
                move = heapq.heappop(moves)[2]
                new_evaluation = self.incremental_evaluate(
                    board, current_evaluation, move
                )
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    new_evaluation,
                    alpha,
                    minEvaluation,
                )
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

Die Funktion `get_next_middle_game_move` wurde um eine `for`-Schleife zur iterativen Tiefensuche erweitert.
Mithilfe dieser Schleife werden die Züge beginnend bei der Tiefe `1` bis zum konfigurierten Limit `DEPTH` berechnet. Beim jeweils nächsten Aufruf kann dann die vorherige Evaluierung aus dem Cache genutzt werden, um die Züge anhand ihrer Evaluierung zu sortieren und somit das Pruning signifikant zu erhöhen.

In [ ]:
import sys


class Exercise07AI(Exercise07AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["cache_hits"] = 0

        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            last_move = board.pop()
            current_evaluation = self.incremental_evaluate(board, self.last_evaluation, last_move)  # type: ignore
            board.push(last_move)

        self.stats[-1]["nodes"] = 0
        for depth in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            future_evaluation, best_move = self.minimax(
                board, depth, current_evaluation
            )

        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = self.incremental_evaluate(
            board, current_evaluation, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation

        if board.is_irreversible(best_move):
            self.cache.clear()
            self.stats[-1]["cache_cleared"] = True
        else:
            self.stats[-1]["cache_cleared"] = False
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise04AI as Exercise04AI_
import Exercise02AI as Exercise02AI_

In [ ]:
# Create player and board
unit_test_player = Exercise07AI(player_name="Ex07AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

Um zu validieren, dass das implementierte Progressive Deepening die Evaluierung des berechneten Ergebnis nicht verändert hat, wird die Minimax-Testfunktion der `Exercise04AI` aufgerufen, wobei dieselben Evaluierungsparameter wie in den vorhergehenden KI-Versionen verwendet werden. Die Anzahl der erwarteten Cache-Anfragen und berechneten Knotenpunkten ist durch das Progressive Deepening höher als in der `Exercise06AI`, durch das verwendete Alpha-Beta-Pruning allerdings niedriger als in der `Exercise04AI`.

In [ ]:
# Test minimax
Exercise04AI_.test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=5553,
    expected_cache_tries=7487,
    expected_cache_hits=2560,
    expected_cache_elements=4927,
)

Um die Funktion `get_next_middle_game_move` zu testen, wird überprüft, ob diese korrekterweise den Zug zurückgibt, in welchem die Dame durch den schwarzen Bauern geschlagen wird. Durch das Progressive Deepening steigt die Zahl der berechneten Knoten auf $81$.

In [ ]:
# Test next move function (with memoized minimax result)
Exercise02AI_.test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=81,
)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.